# Data Prep Script
## School Statistics

In [109]:
import pandas as pd
from pandasql import sqldf
import numpy as np

In [110]:
crime = pd.read_csv('./datasets/school_crime.csv')
crime = crime[['DBN', 'Borough', 'Register', 'Major N', 'Oth N', 'NoCrim N', 'Prop N', 'Vio N']]
crime

,DBN,Borough,Register,Major N,Oth N,NoCrim N,Prop N,Vio N
0,01M301,M,99,NaN,NaN,NaN,NaN,NaN
1,05M357,M,135,0.0,0.0,3.0,0.0,0.0
2,22K052,K,757,1.0,1.0,0.0,2.0,0.0
3,28Q008,Q,336,NaN,NaN,NaN,NaN,NaN
4,75M162,M,SCHOOL CLOSED,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1914,NaN,X,1077,1.0,4.0,22.0,1.0,4.0
1915,07X221,X,578,NaN,NaN,NaN,NaN,NaN
1916,08X100,X,563,1.0,2.0,2.0,1.0,1.0
1917,NaN,X,688,1.0,1.0,13.0,2.0,0.0


In [116]:
crime['Register'] = crime['Register'].replace('GED PROGRAM', np.nan)
crime['Register'] = crime['Register'].replace('EVENING SCHOOL', np.nan)
crime['Register'] = crime['Register'].replace('SCHOOL CLOSED', np.nan)
crime['Register'] = crime['Register'].replace('LEARNING CENTER', np.nan)
crime['Register'] = crime['Register'].replace('ANNEX', np.nan)
crime['Register'] = crime['Register'].replace('YABC', np.nan)

# crime['Register'] = crime['Register'].replace(np.nan, None, regex=True)
# crime['Major N'] = crime['Major N'].replace(np.nan, None, regex=True)
# crime['Oth N'] = crime['Oth N'].replace(np.nan, None, regex=True)
# crime['NoCrim N'] = crime['NoCrim N'].replace(np.nan, None, regex=True)
# crime['Prop N'] = crime['Prop N'].replace(np.nan, None, regex=True)
# crime['Vio N'] = crime['Vio N'].replace(np.nan, None, regex=True)
# crime['DBN'] = crime['DBN'].replace(np.nan, None, regex=True)
crime['Register'] = pd.to_numeric(crime['Register'])


In [117]:
crime['Register'] = pd.cut(crime['Register'], 3, labels=['low - registration', 'med - registration', 'high - registration'])
crime['Major N'] = pd.cut(crime['Major N'], 3, labels=['low - major crimes', 'med - major crimes', 'high - major crimes'])
crime['Oth N'] = pd.cut(crime['Oth N'], 3, labels=['low - other crimes', 'med - other crimes', 'high - other crimes'])
crime['NoCrim N'] = pd.cut(crime['NoCrim N'], 3, labels=['low - noncriminal crimes', 'med - noncriminal crimes', 'high - noncriminal crimes'])
crime['Prop N'] = pd.cut(crime['Prop N'], 3, labels=['low - property crimes', 'med - property crimes', 'high - property crimes'])
crime['Vio N'] = pd.cut(crime['Vio N'], 3, labels=['low - violent crimes', 'med - violent crimes', 'high - violent crimes'])


<bound method NDFrame.describe of          DBN Borough            Register              Major N  \
0     01M301       M  low - registration                  NaN   
1     05M357       M  low - registration   low - major crimes   
2     22K052       K  low - registration   low - major crimes   
3     28Q008       Q  low - registration                  NaN   
4     75M162       M                 NaN                  NaN   
...      ...     ...                 ...                  ...   
1914     NaN       X  low - registration   low - major crimes   
1915  07X221       X  low - registration                  NaN   
1916  08X100       X  low - registration   low - major crimes   
1917     NaN       X  low - registration   low - major crimes   
1918     NaN       Q  low - registration  high - major crimes   

                   Oth N                  NoCrim N                 Prop N  \
0                    NaN                       NaN                    NaN   
1     low - other crimes  low -

**Note** : There is no point in ignoring data with NaN since we are trying to find association rules at the element level, not the column level (i.e. we are not interested in finding correlation between 2 data variables but are interesting in finding frequently occuring relationships). Hence, ignoring NaN (replacing with an empty string ``) makes more sense.
**Bucketing Data** : Since the above data is mostly numerical, we bucket the following columns:
- `Register` : to values [`~x students`]

In [124]:
attendance = pd.read_csv('./datasets/school_attendance.csv')
attendance['Present'] = attendance['Present'] / attendance['RosterCount']
attendance['Absent'] = attendance['Absent'] / attendance['RosterCount']
attendance = sqldf('SELECT School, AVG(Absent) as "avg_absent", AVG(Present) as "avg_present" FROM attendance GROUP BY School')
attendance

,School,avg_absent,avg_present
0,01M015,1.113806,16.308639
1,01M019,1.435950,15.999071
2,01M020,1.426859,15.949860
3,01M034,1.711447,15.588943
4,01M063,1.237471,16.280718
...,...,...,...
1608,75X352,2.455589,14.778044
1609,75X469,2.079392,15.392551
1610,75X721,2.466649,14.961378
1611,75X754,6.438527,10.060969


In [126]:
attendance['avg_absent'] = pd.cut(attendance['avg_absent'], 3, labels=['low - absent', 'med - absent', 'high - absent'])
attendance['avg_present'] = pd.cut(attendance['avg_present'], 3, labels=['low - present', 'med - present', 'high - present'])

In [128]:
final = sqldf("SELECT * FROM crime, attendance WHERE crime.DBN = attendance.School")
final.to_csv('school_records.csv')

## 

In [131]:
film = pd.read_csv('./datasets/film.csv')
film = film[['EventType', 'Borough', 'Category', 'SubCategoryName', 'ZipCode(s)', 'Country']]
film.to_csv('./datasets/mfilm.csv')

##


In [139]:
lcd = pd.read_csv('./datasets/nyclcd.csv')
lcd = lcd[['Leading Cause', 'Sex', 'Race Ethnicity', 'Deaths']]
#film.to_csv('./datasets/mfilm.csv')
lcd['Sex'] = lcd['Sex'].replace('Female', 'F')
lcd['Sex'] = lcd['Sex'].replace('Male', 'M')
lcd['Race Ethnicity'] = lcd['Race Ethnicity'].replace('Not Stated/Unknown', None)



,Year,Leading Cause,Sex,Race Ethnicity,Deaths
0,2009,"Nephritis, Nephrotic Syndrome and Nephrisis (N...",F,Other Race/ Ethnicity,.
1,2013,Influenza (Flu) and Pneumonia (J09-J18),F,Hispanic,204
2,2012,"Assault (Homicide: Y87.1, X85-Y09)",M,Other Race/ Ethnicity,.
3,2007,Essential Hypertension and Renal Diseases (I10...,F,Not Stated/Unknown,5
4,2014,Cerebrovascular Disease (Stroke: I60-I69),F,White Non-Hispanic,418
...,...,...,...,...,...
1267,2019,Influenza (Flu) and Pneumonia (J09-J18),F,Not Stated/Unknown,8
1268,2019,Certain Conditions originating in the Perinata...,F,Not Stated/Unknown,7
1269,2019,Mental and Behavioral Disorders due to Acciden...,F,Not Stated/Unknown,5
1270,2019,"Intentional Self-Harm (Suicide: U03, X60-X84, ...",F,Not Stated/Unknown,5
